# **Génération des sets de données T pour la phase d'Inversion**<br/>sauvedarde en DataFrame

N-Notebook `generate_INVERSION_data_sets_to_DF_vPL0.ipynb`

<br/>

Notebook pour générer les sets de données à utiliser pour la deuxième phase : le processus d'inversion des profils de Température. Pour la génération des données pour l'entrainement des réseaux voir le Notebook `generate_TRAIN-TEST_data_sets_to_DF_vPL0.ipynb`

Utilise les fichiers de données `v4` des simulations pour des différents forçages qui forcent les modèles: `GHG`, `AER`, `NAT` et `HIST`, et pour 13 les modèles océanographiques.  Ces fichiers sont stockés en format Pickle de Python.

Pour l'INVERSION, les ensembles de données sont formés uniquement par les combinaison des simulations des trois forçages `GHG`, `AER` et `NAT` par modèle. Le forçage HIST n'intervienne pas dans ce  ensemble de données.

___
Voici les fichiers qu'interviennent pour la générations des data sets:

- `ALL_sim_v4.p`, contenant un array 2D avec les dimensions des données par Modèle (colonne, voir ci-dessous) et par forçage (ligne, dans l'ordre: `GHG`, `AER`, `NAT`, `HIST` et `other??`). Cet array sera lu dans la variable `all_sim`.<br/>Voici son contenu:

>                0    1    2    3    4    5    6    7    8    9   10   11   12
>              ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---
>     GHG:  [[   3    3   50   10    3   10    3    4    5   10    3    3    1]
>     AER:   [   3    3   30   10    3   10   10    4    5   15    2    3    1]
>     NAT:   [   3    3   50   10    3   10   44    4    5   20    3    3    3]
>     HIST:  [   3    6   65   30   40   33   50    5   11   35   11    3    3]
>     ??:    [ 600  700 1000  500  500 1000  500  500  500  851  500  110  500]]
    
- `Models_v4.p`, fichiers contenant la liste des noms des 13 modèles climatiques, dans l'ordre des 13 colonnes de l'array dans `ALL_sim_v4.p` et lue dans la variable `all_models`. Voici une représentation en tableau de cette liste:
> 0- `'BCC-CSM2-MR'`,<br/>
> 1- `'FGOALS-g3'`,<br/>
> 2- `'CanESM5'`,<br/>
> 3- `'CNRM-CM6-1'`,<br/>
> 4- `'ACCESS-ESM1-5'`,<br/>
> 5- `'IPSL-CM6A-LR'`,<br/>
> 6- `'MIROC6'`,<br/>
> 7- `'HadGEM3-GC31-LL'`,<br/>
> 8- `'MRI-ESM2-0'`,<br/>
> 9- `'GISS-E2-1-G'`,<br/>
> 10- `'CESM2'`,<br/>
> 11- `'NorESM2-LM'`,<br/>
> 12- `'GFDL-ESM4'`,


- `T_ghg_v4.p`, `T_aer_v4.p`, `T_nat_v4.p` et `T_hist_v4.p`, fichiers contenant les simulations pour tous les modèles séparés par type de forçage. Les modèles sont dans l'ordre du fichier `Models_v4.p`.  Une simulation est un array 1D de 165 valeurs: les valeurs étant les moyennes annuelles de T pour les 165 années de 1850 à 2014.

___
## Tableau des Modèles et combinaisons des simulations

| n | MODEL           | simulations G-A-N-H | combi.G-A-N | combi.G-A-N-H |
|--:|---              |       :---        |     ---:|     ---:|
| 0)| BCC-CSM2-MR     | [3, 3, 3, 3]      |     27  |      81 |
| 1)| FGOALS-g3       | [3, 3, 3, 6]      |     27  |     162 |
| 2)| CanESM5         | [50, 30, 50, 65]  |  75000  | 4875000 |
| 3)| CNRM-CM6-1      | [10, 10, 10, 30]  |   1000  |   30000 |
| 4)| ACCESS-ESM1-5   | [3, 3, 3, 40]     |     27  |    1080 |
| 5)| IPSL-CM6A-LR    | [10, 10, 10, 33]  |   1000  |   33000 |
| 6)| MIROC6          | [3, 10, 44, 50]   |   1320  |   66000 |
| 7)| HadGEM3-GC31-LL | [4, 4, 4, 5]      |     64  |     320 |
| 8)| MRI-ESM2-0      | [5, 5, 5, 11]     |    125  |    1375 |
| 9)| GISS-E2-1-G     | [10, 15, 20, 35]  |   3000  |  105000 |
|10)| CESM2           | [3, 2, 3, 11]     |     18  |     198 |
|11)| NorESM2-LM      | [3, 3, 3, 3]      |     27  |      81 |
|12)| GFDL-ESM4       | [1, 1, 3, 3]      |      3  |       9 |

___
C.Mejia IPSL/LOCEAN `carlos.mejia@locean.ipsl.fr`<br/>
Juin 2022

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pickle
import os
import itertools
import datetime
import os, sys
import pandas as pd


In [34]:
# To load personal libraries (currently in folder 'tools'):
sys.path.append('tools')

#import generic_tools_pl as gt   # like hexcolor(), ...
#import specific_nn_tools_pl as nnt  # like CustomDataset(), Net(), train_model() 
import generic_tools_pl as gt
from generic_tools_pl import region

In [35]:
gt.region="48"
region=gt.region
print(gt.region)

48


In [36]:
# Call set_printoptions(threshold=x) to change the threshold where arrays are truncated. To prevent truncation, set the threshold to np.inf.
# Change linewidth to print wider lines ...
#display(np.get_printoptions())
np.set_printoptions(threshold=np.inf,linewidth=180) # good for a 1920 pixels wide screen
pd.set_option('display.max_columns', 18)            # 
#np.set_printoptions(threshold=np.inf,linewidth=240)   # good for an iMac 27" screen
#pd.set_option('display.max_columns', 100)             # (this is maybe too much!)

___
## Conditions pour former les ensembles de données pour le processus d'INVERSION


### Génération des Données Méthode 1 - INVDGM1: _Combinaisons G-A-N-H Random_
>Pour les données d'Inversion, on se base sur les combinaisons des quatre profils  G-A-N-H (GHG, AER, NAT, HOST).
>
>Sans repetition, on prend un nombre maximum de combinaisons, limite fixé par `n_sim_max_by_mod`.
>
>A chaque traitement de la boucle (chaque modele) le générateur de nombres aléatoires est initialisé selon la variable `seed_for_combi`.
### Identification des données générées

Les variables `seed_for_combi`, `train_part` et `n_sim_for_train_by_mod` servent à identifier les données générées concernant l'ensemble TRAIN. L'ensemble TEST est quant a lui identifié aussi par `seed_for_combi`, `train_part` mais en troisième c'est `n_sim_max_for_test_by_mod` qu'intervient.

En effet, deux labels nommés `train_set_suffix` et `test_set_suffix` sont créés en fonction de ces variables et est utilise comme partie du nom des fichiers où leur données respectives seront stockés. On ajoute `v4` aux labels étant, _a priori_ la version des données source.

```python
inversion_suffix = f"{data_gener_method}-S{seed_for_combi}-NMx{n_sim_max_by_mod}_v4"

# Exemples:
#  La declaration des variables:
data_gener_method = 'INVDGM1'
seed_for_combi = 1
n_sim_max_by_mod = 100

#  Donne l'identifiant:
#    'INVGDM1-S1-NMx100_v4'
```

<br/>

___
### Déclaration des variables determinantes pour la generation des données d'INVERSION et pour leur identification

_Voir explication ci-dessus._

In [37]:
# Label identifiant la méthode de génération des données
data_gener_method = 'INVDGM1'  # Combinaisons G-A-N-H Random

#seed_for_combi = 0    # seed 0 utilisée pour TRAIN-TEST, je prefere utiliser un autre ... 
seed_for_combi = 1    # "semence" d'initialisation du générateur de chiffres aléatoires

# nombre de données (combinaisons GANH) tirées par modèle, maximum (decommenter le choix)
n_sim_max_by_mod = 100
#n_sim_max_by_mod = 400
#n_sim_max_by_mod = 1000

#--------------------
dataframe_format_ok = True
array_format_ok = False
#--------------------
save_anc_data_only_ok = True  # if TRUE saves a dictionary with index, number by mod, list of models, of forcings, of years. but not data arrays
#save_anc_data_only_ok = False
#--------------------
save_data_ok = True
#--------------------
#rewrite_files_ok = True
rewrite_files_ok = False


### Forçages et Modèles participant aux données à générer

La declaration des listes `forcing_names` et `model_names` permet de sélectionner les forçages et modèles à prendre en compte (ou rejeter si absent dans la liste) dans le processus de génération des données.

Eventuellement, autorise aussi des changements d'ordre dans la specification des forçages et/ou des modèles.

Ces listes de forçages et modèles seront associées par les variables `forcing_index` et `model_index` aux listes source `all_models_src` et `all_forcings_src`, représentant l'ordre dans les fichiers de données source.  Ces variables, `forcing_index` et `model_index`, ayant les indices se référant aux listes source mais aussi au tableau du nombre de simulations dans `all_sim_src`.

In [38]:
# -----------------------------------------------------------------------------------------------
# Selection of working Forcing and Models:
# -----------------------------------------------------------------------------------------------

# list of selected forcings
#forcing_names = [ 'ghg', 'aer', 'nat', 'hist' ]
#forcing_color = [ 'r',   'b',   'g',   'k' ]

forcing_names = [ 'ghg', 'aer', 'nat' ]   # 'hist' n'intervient pas dans l'ensemble de données INVERSION
forcing_color = [ 'r',   'b',   'g' ]


# list of selected models 
model_names = ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CNRM-CM6-1', 'CanESM5', 'FGOALS-g3', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0', 'NorESM2-LM']

___
<br/><br/>

In [39]:
#!echo " --- donnees au Locean (sur ether) /net/ether/data/varclim/ggalod/Constantin/:"
#!ls -rtl /net/ether/data/varclim/ggalod/Constantin/ 

In [40]:
#!echo " --- donnees sur jean-zay:/gpfswork/rech/ryn/rces866/Constantin/:"
#!ssh uvc15wk@jean-zay.idris.fr ls -lrt /gpfswork/rech/ryn/rces866/Constantin/

In [41]:
#!echo " --- donnees sur Ciclad"
#!ls -rtl /scratchu/ggalod/Constantin

In [42]:
#!echo " --- /datatmp/data/Constantin/:"
#!ls -l /datatmp/data/Constantin/

In [43]:
#verbose = False
verbose = True
#--------------------
save_figs = True
#save_figs = False
#--------------------
figs_dir = 'Figs'

local_fig_label = "Nb1_gener_INV_data_set_to_df"   # label a appliquer aux noms des fichiers (figures) produits dans le notebook

figs_defaults = { 'dpi' : 300, 'facecolor' : 'w', 'edgecolor' : 'w'} # ajuter format='png' ou autre a l'appel de savefig()

if save_figs and not os.path.exists(figs_dir):
    os.makedirs(figs_dir)


In [44]:
# ether_dir = '/net/ether/data/varclim/ggalod/Constantin'

#data_dir = '/datatmp/data/constantin_data/data_source'
#data_dir = '/net/acratopotes/datatmp/data/constantin_data/data_source'
#data_dir = 'data'

# Repertoire des donnees
try:
    # WORK dir carlos projet ryn sur Jean Zay
    data_dir = '/Users/dahmani/Desktop/stagelong/projetlong/data_source_dr/Region{}'.format(region)
    if not os.path.isdir(data_dir):
        print(f" ** data_dir '{data_dir}' not found. Trying next...")
        
        # WORK dir Guillaume sur Jean Zay ** NON **
        #data_dir = '/gpfswork/rech/ryn/rces866/Constantin'
        #if not os.path.isdir(data_dir):
        #print(f" ** data_dir '{data_dir}' not found. Trying next...")
        
        # SSD sur Acratopotes au Locean
        data_dir = '/net/acratopotes/datatmp/data/constantin_data/data_source_pl'
        if not os.path.isdir(data_dir):
            print(f" ** data_dir '{data_dir}' not found. Trying next...")
            
            # sur Cloud SU (carlos)
            data_dir = os.path.expanduser('~/Clouds/SUnextCloud/Labo/Travaux/Theses-et-stages/These_Constantin/constantin_data/data_source_pl')
            if not os.path.isdir(data_dir):
                print(f" ** data_dir '{data_dir}' not found. Trying next...")
                
                # en dernier recours, en esperant qu'il y a un repertoire 'data' present ...
                data_dir = os.path.expanduser('data')
                if not os.path.isdir(data_dir):
                    print(f" ** data_dir '{data_dir}' not found at all **\n")
                    raise Exception('data_dir not found')

except Exception as e:
    print(f'\n *** Exception error "{e}" ***\n')
    raise

print(f"data_dir found at '{data_dir}'")

#dataframe_ok = True
dataframe_ok = False

data_dir found at '/Users/dahmani/Desktop/stagelong/projetlong/data_source_dr/Region48'


In [45]:
#!ls -l /net/acratopotes/datatmp/data/constantin_data/

## Variables derivées des données source (v4)

Mise au point des variables:

- `all_sim`, array contenant le nombre de simulations par forcage et par modèle (fichier "ALL_sim_v4.p").<br/>
Le nombre de simutations par modèle dans ce array correspond aux simulations présentes, dans l'ordre, dans les fichiers "T_{*fff*}_v4.p" (à remplacer {*fff*} par chacun des quatre forçages `ghg`, `aer`, `nat` et `hist`).

- `all_models`, liste des noms des modeles, dans l'ordre des colonnes de `all_sim` (fichier "Models_v4.p").
- `all_forcings`, liste des noms des forçages (liste composée manuellement selon l'ordre des lignes de l'array `all_sim`).<br/>
Mis à part les forçages classiques `ghg`, `aer`, `nat` et `hist`, l'array `all_sim` contient cinq lignes dont pour l'instant je ne connait pas la signification de la derniere ligne. J'ai appellé ce "cinquième" forgage `other`. Cependant, pour l'ensemble d'INVERSION on n'utilise que les trois forcages `ghg`, `aer` et `nat` representant les etat initiaux des processus d'inversion. La sortie pour les CNN étant les observations.

### Lecture des fichiers source

#### _Fichiers de dimensions (nombre de simulations disponibles par modèle et par forçage) et des noms des modèles (v4)_

In [46]:
def load_data_and_gener_df(models=None, forcings=None, 
                           data_dir='.', sim_file="All_sim_z{}v0.p".format(region), models_file="models_z{}v0.p".format(region), 
                           forcings_src=[ 'ghg', 'aer', 'nat', 'hist', 'other' ], verbose=False):
    print(f"\nLecture des meta donnees (ALL_sim, liste de modeles, ...) ...")

    # ALL_sim_v4 et Models_v4:
    _all_sim_src  = pickle.load( open( os.path.join(data_dir,"All_sim_z{}v0.p".format(region)), "rb" ), encoding="latin1")

    # liste de modeles
    _all_models_src  = pickle.load( open( os.path.join(data_dir,"models_z{}v0.p".format(region)), "rb" ), encoding="latin1")
    
    # forcages dans l'ordre des lignes de _all_sim_src.
    # l'ordre des forcages est trouve en comparant le nombre de simulations par forcage (tout modele)
    # avec les dimensions des donnees dans les fichiers "T_xxx_v4.p", pour les differents forcages xxx
    _all_forcings_src = forcings_src

    # list of years in source files 
    years = np.arange(1850,2015)

    if models is None :
        models = _all_models_src

    if forcings is None :
        models = forcings_src[:-1] # tous sauf le dernier

    # tableau d'indices de debut et de fin des donnees du modele (colonne) pour chaque forcage (ligne)
    simulation_ini_index_list = np.cumsum(_all_sim_src,axis=1) - _all_sim_src
    simulation_end_index_list = np.cumsum(_all_sim_src,axis=1)

    if verbose:
        print("\nContenu de 'ALL_sim (v4).p':")
        print(_all_sim_src)
        print("Dim:",_all_sim_src.shape)
        print(f'\n_all_models_src: {len(_all_models_src)} models (v4) :\n  {_all_models_src}')
        print(f'\n_all_forcings_src: {len(_all_forcings_src)} forcings :\n  {_all_forcings_src}')
        print('\n Somme par colonne:',np.sum(_all_sim_src,axis=0))  #,"    (je suppose, c'est le nombre de simulations par modele, pour les 4+1 forcages)")
        print('\n Somme par ligne:  ',np.sum(_all_sim_src,axis=1))  #,"    (je suppose, c'est le nombre de simulations par forcace, tout modele confondu)" )
        print("\nIndices INI et END des tableaux des donnees dans fichiers 'T_xxx_v4.p' ('xxx' etant le forcage):")
        print(f"Indices INI:\n{simulation_ini_index_list}")
        print(f"Indices INI:\n{simulation_end_index_list}")

    print(f"\nLecture des donnees des forcages ...")

    list_of_df = []
    for iforcing,forcing in enumerate(forcings) :
        if verbose:
            print(f" {forcing.upper()}",end='')
        kfor = _all_forcings_src.index(forcing)
        current_t_array_filename = f"T_{forcing.lower()}_z{region}v0.p"
        T_xxx  = pickle.load(open( os.path.join(data_dir,current_t_array_filename), "rb" ),
                             encoding="latin1")
        if verbose:
            #print(T_xxx)
            print(f" Data '{current_t_array_filename}' -> dim: {T_xxx.shape}")

        #current_t_df_varname = f'T_{forcing.lower()}_df'
        xxx_df = pd.DataFrame(T_xxx, columns=years)

        xxx_df['model'] = '<NA>'   # ajoute colonne 'model'
        cols = xxx_df.columns.tolist()
        xxx_df = xxx_df[cols[-1:] + cols[:-1]]    # remet la colonne 'model' en premier
        #yyy_df = xxx_df.copy()
        for imod,mod in enumerate(models) :
            kmod = _all_models_src.index(mod)
            ii = simulation_ini_index_list[kfor,kmod]
            jj = simulation_end_index_list[kfor,kmod]
            xxx_df.loc[np.arange(ii,jj),'model'] = mod
            if verbose:
                print(f"   {imod}) '{mod}' -> {ii}, {jj} ({jj-ii})")
        # retire les lignes non identifiees par un modele
        xxx_df = xxx_df.loc[lambda df: df['model'] != '<NA>', :]
        #globals()[current_t_df_varname] = xxx_df
        #print(f" ... assigned to DataFrame '{current_t_df_varname}'")
        list_of_df.append(xxx_df)
    
    return models, forcings, years, list_of_df


#### _Lecture des fichiers des profils Temperature par forçage (v4)_

In [47]:
#def load_data_and_gener_df(models=None, forcings=None, 
#                           data_dir='.', sim_file="ALL_sim_v4.p", models_file="Models_v4.p", 
#                           forcings_src=[ 'ghg', 'aer', 'nat', 'hist', 'other' ], verbose=False):
#    ...
#    return models, forcings, list_of_df

_, _, years, (T_ghg_df,T_aer_df,T_nat_df) = load_data_and_gener_df(models=model_names, forcings=forcing_names, data_dir=data_dir, verbose=verbose)



Lecture des meta donnees (ALL_sim, liste de modeles, ...) ...

Contenu de 'ALL_sim (v4).p':
[[ 3  3  3  8 50  3 10  4  8  3  5  3]
 [ 3  3  2  8 30  3 10  4  8 10  5  3]
 [ 3  3  3  8 50  3 20  4  8 50  5  3]
 [15  3 11 28 65  6 28  5 31 50 11  3]]
Dim: (4, 12)

_all_models_src: 12 models (v4) :
  ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CNRM-CM6-1', 'CanESM5', 'FGOALS-g3', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0', 'NorESM2-LM']

_all_forcings_src: 5 forcings :
  ['ghg', 'aer', 'nat', 'hist', 'other']

 Somme par colonne: [ 24  12  19  52 195  15  68  17  55 113  26  12]

 Somme par ligne:   [103  89 160 256]

Indices INI et END des tableaux des donnees dans fichiers 'T_xxx_v4.p' ('xxx' etant le forcage):
Indices INI:
[[  0   3   6   9  17  67  70  80  84  92  95 100]
 [  0   3   6   8  16  46  49  59  63  71  81  86]
 [  0   3   6   9  17  67  70  90  94 102 152 157]
 [  0  15  18  29  57 122 128 156 161 192 242 253]]
Indices INI:
[[  3   6   9  17  

In [15]:
display(T_ghg_df.head())
display(T_aer_df.head())
display(T_nat_df.head())
#display(T_ghg_df.drop('model', axis=1).head()) # retire la colonne 'model' et affiche les 5 premieres lignes uniquement


,model,1850,1851,1852,1853,1854,1855,1856,1857,...,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,ACCESS-ESM1-5,0.431712,0.348646,0.271168,0.203422,0.148339,0.107325,0.080210,0.065426,...,1.274852,1.262540,1.246546,1.225753,1.199389,1.167242,1.129778,1.088150,1.044075
1,ACCESS-ESM1-5,-0.368239,-0.335154,-0.301370,-0.266712,-0.231237,-0.195237,-0.159178,-0.123588,...,1.423819,1.380972,1.309702,1.207211,1.072980,0.909084,0.720199,0.513281,0.296939
2,ACCESS-ESM1-5,-0.114676,-0.052192,0.006851,0.059287,0.102428,0.134292,0.153762,0.160644,...,1.397555,1.431325,1.479136,1.542792,1.622858,1.718479,1.827358,1.945910,2.069582
3,BCC-CSM2-MR,-0.362714,-0.327733,-0.293884,-0.262191,-0.233334,-0.207564,-0.184701,-0.164199,...,1.083175,1.102747,1.112693,1.110902,1.096278,1.068912,1.030090,0.982152,0.928212
4,BCC-CSM2-MR,-0.073991,-0.033453,0.005804,0.041749,0.072672,0.097369,0.115238,0.126286,...,1.148506,1.147078,1.127923,1.089554,1.031903,0.956378,0.865779,0.764078,0.656057


,model,1850,1851,1852,1853,1854,1855,1856,1857,...,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,ACCESS-ESM1-5,0.201333,0.191304,0.179779,0.165896,0.149068,0.129082,0.106200,0.081210,...,-0.937590,-0.881635,-0.799219,-0.691547,-0.561202,-0.412022,-0.248888,-0.077450,0.096225
1,ACCESS-ESM1-5,-0.256948,-0.229524,-0.201290,-0.172280,-0.142670,-0.112750,-0.082907,-0.053627,...,-0.602661,-0.573246,-0.550727,-0.538143,-0.537448,-0.549300,-0.573027,-0.606765,-0.647736
2,ACCESS-ESM1-5,0.258495,0.199671,0.142768,0.089848,0.042806,0.003240,-0.027652,-0.049141,...,-0.781956,-0.752718,-0.721790,-0.691600,-0.664162,-0.640875,-0.622445,-0.608915,-0.599780
3,BCC-CSM2-MR,0.375996,0.331140,0.286148,0.241717,0.198490,0.157032,0.117854,0.081461,...,-0.236864,-0.238039,-0.234444,-0.226289,-0.214115,-0.198686,-0.180878,-0.161579,-0.141623
4,BCC-CSM2-MR,-0.189650,-0.113235,-0.038177,0.033471,0.099742,0.158838,0.209183,0.249467,...,-0.144304,-0.130201,-0.113928,-0.095369,-0.074535,-0.051606,-0.026974,-0.001246,0.024792


,model,1850,1851,1852,1853,1854,1855,1856,1857,...,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,ACCESS-ESM1-5,0.231619,0.143334,0.061757,-0.008045,-0.062230,-0.098497,-0.116215,-0.116372,...,0.019066,0.072678,0.112576,0.136224,0.142466,0.131621,0.105415,0.066771,0.019456
1,ACCESS-ESM1-5,-0.076634,-0.064711,-0.052470,-0.039775,-0.026555,-0.012825,0.001282,0.015480,...,0.096157,0.074520,0.033567,-0.029101,-0.114263,-0.220830,-0.345812,-0.484510,-0.630945
2,ACCESS-ESM1-5,-0.342751,-0.242587,-0.146044,-0.057827,0.018030,0.078508,0.121792,0.147331,...,0.142650,0.131368,0.110485,0.080242,0.041387,-0.004847,-0.056779,-0.112371,-0.169363
3,BCC-CSM2-MR,-0.004337,0.003063,0.012168,0.023674,0.037726,0.053878,0.071152,0.088206,...,-0.007191,0.002119,0.024805,0.061818,0.112949,0.176907,0.251484,0.333779,0.420456
4,BCC-CSM2-MR,-0.436827,-0.326050,-0.218689,-0.118653,-0.029479,0.045955,0.105644,0.148590,...,0.065845,0.067280,0.074174,0.087934,0.109324,0.138366,0.174337,0.215845,0.260988


In [16]:
# profils GHG de l'IPSL
#T_ghg_df.loc[lambda df: df['model'] == 'IPSL-CM6A-LR', :].drop('model', axis=1)

### Traitements

Prépare les données source qui seront combinés pour former des arrays de combinaisons entre simulations de diffeents forcages pour un même modele

#### _Préparation des tableaux d'indices pour conformer les ensembles de données d'INVERSION_

In [17]:
%%time

#verbose = False

# Initialisation des listes retournées par la boucle ---------------------------------------------------------------------
inversion_combi_list  = []   # liste des indices pour composer l'ensemble pour l'INVERSION. 
#
n_inversion_combi_list  = [] # liste du nombre de données pour l'INVERSION par modèle (le nombre de combinaisons GANH limité au max par la valeur dans la variable n_sim_max_by_mod)
# ------------------------------------------------------------------------------------------------------------------------

for imod,mod in enumerate(model_names) :
    #imod = 0
    #imod = 5
    #mod = model_names[imod]

    if verbose :
        print(f"\n{'-'*92}",end='')
    print(f'\n{imod:2d}) {mod:15s} ',end='')

    # initialise le generateur de nombres aléatoires
    print(f"\n Re-initialise le generateur de nombres aleatoires par le SEED: {seed_for_combi}")
    np.random.seed(seed_for_combi)

    # liste de simulations par forcage du modele en cours:
    T_mod_isim_by_forcing = []
    for iforcing,forcing in enumerate(forcing_names) :
        current_t_df_varname = f'T_{forcing.lower()}_df'
        xxx_df = globals()[current_t_df_varname]
        xxc_indices = xxx_df.loc[lambda df: df['model'] == mod].index
        ii = xxc_indices[0]    # indice inferieur (incluyant)
        jj = xxc_indices[-1]+1 # indice superieur (excluyant)
        if verbose:
            print('\n  ',iforcing, forcing, ii, jj, jj-ii,end='')

        # cumule toutes les simulations du modele et forcage en cours
        T_mod_isim_by_forcing.append(np.arange(ii,jj).tolist())
    
    if verbose :
        print(f'\n  Indices des focages GAN (GHG, AER, NAT):')
        [print(f"\n    {len(ksim)} indices pour {forcing_names[ii].upper()}: {ksim}",end='') for ii,ksim in enumerate(T_mod_isim_by_forcing)];
    else:
        print(f'\n  Indices des focages ',end='')
        [print(f"  {len(ksim)} pour {forcing_names[ii].upper()},",end='') for ii,ksim in enumerate(T_mod_isim_by_forcing)];
        print('')

    all_forcings_combi = list(itertools.product(*T_mod_isim_by_forcing))
    n_all_forcings_combi = len(all_forcings_combi)
    
    if verbose :
        n_to_show = int(np.min([n_all_forcings_combi/2, 9]))
        print(f"\n  {n_all_forcings_combi} combinaisons d'indices GAN:")
        print('   ',*all_forcings_combi[:n_to_show], ' ...\n     ', *all_forcings_combi[-n_to_show:])
    
    # ------------------------------------------------------------------------------------------------------------------------
    # INVERSION Set:
    # Nombre variable de patterns (combinaisons G-A-N-H) par modèle. Pas de repetitions. Toutes les combinaisons G-A-N-H sont
    # prise en compte mais avec un nombre max limite. Combinaisons choisies aleatoirement uniquement dans le cas ou il faut
    # restreindre leur nombre.
    # ------------------------------------------------------------------------------------------------------------------------

    if n_all_forcings_combi > n_sim_max_by_mod :
        permuted_inversion_all_combi = np.random.permutation(all_forcings_combi).tolist()
        inversion_all_combi = permuted_inversion_all_combi[:n_sim_max_by_mod]
    
        print(f"\n ** Restricting to {len(inversion_all_combi)} over {n_all_forcings_combi} sim. combinations for INVERSION set for {mod}.")
    else:
        inversion_all_combi = all_forcings_combi
        print(f"\n  - Selecting all {len(inversion_all_combi)} sim. combinations for INVERSION set for {mod}.")
    
    inversion_combi_list += inversion_all_combi
    n_inversion_combi_list += [len(inversion_all_combi)]
    


--------------------------------------------------------------------------------------------
 0) ACCESS-ESM1-5   
 Re-initialise le generateur de nombres aleatoires par le SEED: 1

   0 ghg 0 3 3
   1 aer 0 3 3
   2 nat 0 3 3
  Indices des focages GAN (GHG, AER, NAT):

    3 indices pour GHG: [0, 1, 2]
    3 indices pour AER: [0, 1, 2]
    3 indices pour NAT: [0, 1, 2]
  27 combinaisons d'indices GAN:
    (0, 0, 0) (0, 0, 1) (0, 0, 2) (0, 1, 0) (0, 1, 1) (0, 1, 2) (0, 2, 0) (0, 2, 1) (0, 2, 2)  ...
      (2, 0, 0) (2, 0, 1) (2, 0, 2) (2, 1, 0) (2, 1, 1) (2, 1, 2) (2, 2, 0) (2, 2, 1) (2, 2, 2)

  - Selecting all 27 sim. combinations for INVERSION set for ACCESS-ESM1-5.

--------------------------------------------------------------------------------------------
 1) BCC-CSM2-MR     
 Re-initialise le generateur de nombres aleatoires par le SEED: 1

   0 ghg 3 6 3
   1 aer 3 6 3
   2 nat 3 6 3
  Indices des focages GAN (GHG, AER, NAT):

    3 indices pour GHG: [3, 4, 5]
    3 indices pou

In [18]:
#inversion_combi_list
n_inversion_combi_list

[27, 27, 18, 100, 100, 27, 100, 64, 100, 100, 100, 27]

In [19]:
# conversion en array d'indices les listes
inversion_combi_index = np.array(inversion_combi_list)    # indices pour composer l'ensemble d'INVERSION. 
#
n_inversion_combi = n_inversion_combi_list       # nombre de données en INVERSION par modèle (le nombre de combinaisons GANH limité au max par la valeur dans la variable n_sim_max_by_mod)

#### _Sauvegarde des tableaux d'indices et du nombre de simulations par modèle pour les ensembles de TRAIN et TEST_

In [20]:
def save_data_file(data, filename, data_dir='.', label='data', rewrite=False, df=False):
                   
    current_filename = os.path.join(data_dir,filename)

    if (os.path.isfile(current_filename) and rewrite) or not os.path.isfile(current_filename) :
        if os.path.isfile(current_filename) :
            print(f" Saving current {label} to filename '{filename}' ... ** File exists: REWRITED **")
        else:
            print(f" Saving current {label} to filename '{filename}'")
        if df :
            data.to_pickle(os.path.join(data_dir,filename) )
        else:
            pickle.dump(data, open( os.path.join(data_dir,filename), "wb" ))
    else:
        if os.path.isfile(current_filename) and not rewrite :
            print(f" ** Warning: {label} file '{filename}' already exists, saving is not possible **\n ** in directory '{data_dir}'\n ** You can authorize rewriting saying True to 'rewrite' option.\n")


In [21]:
print(f"\nEcriture sous forme de dictionaire, pour INVERSION, de:\n"+\
      f" - tableaux d'indices,\n"+\
      f" - liste du nombre de forcages par modèle,\n"+\
      f" - liste de modeles,\n"+\
      f" - liste de forcages:\n")

print(f"\nEcriture des tableaux d'indices et du nombre de forcages par modèle:\n")

if save_data_ok :

    inversion_suffix = f"{data_gener_method}-S{seed_for_combi}-NMx{n_sim_max_by_mod}_z{region}v0"

    current_inversion_dic = { 'index' : inversion_combi_index,
                              'n_by_mod' : n_inversion_combi,
                              'models' : model_names,
                              'forcings' : forcing_names,
                              'years' : years }

    current_inversion_combi_index_filename = f"inversion-combi-dic_{inversion_suffix}.p"
    
    save_data_file(current_inversion_dic, current_inversion_combi_index_filename, data_dir=data_dir,
                   label='inversion combi dictionary', rewrite=rewrite_files_ok)

else:
    print(f"\n ** data saving not authorized: set True to `save_data_ok` anBoolean variable **\n")



Ecriture sous forme de dictionaire, pour INVERSION, de:
 - tableaux d'indices,
 - liste du nombre de forcages par modèle,
 - liste de modeles,
 - liste de forcages:


Ecriture des tableaux d'indices et du nombre de forcages par modèle:

 ** Warning: inversion combi dictionary file 'inversion-combi-dic_INVDGM1-S1-NMx100_z48v0.p' already exists, saving is not possible **
 ** in directory '/Users/dahmani/Desktop/stagelong/projetlong/data_source_dr/Region48'
 ** You can authorize rewriting saying True to 'rewrite' option.



#### _Sauvegarde des données par forçage des simulations issues des combinaisons réalisées dans la composition de l'ensemble d'INVERSION_

In [28]:
save_anc_data_only_ok = False

In [29]:
print(f"\nEcriture des donnees des forcages:\n")

if save_data_ok and not save_anc_data_only_ok:

    for iforcing,forcing in enumerate(forcing_names) :
        print(f"\n{forcing.upper()}:")
        
        current_t_df_varname = f'T_{forcing.lower()}_df'
        xxx_df = globals()[current_t_df_varname]

        current_inversion_index = inversion_combi_index[:,iforcing]

        if dataframe_format_ok :
            current_inversion_filename = f"inversion-{forcing.lower()}_{inversion_suffix}_df.p"

            inversion_df = xxx_df.loc[current_inversion_index]
            inversion_df.reset_index(inplace=True, drop=True)

            save_data_file(inversion_df, current_inversion_filename, data_dir=data_dir,
                           label=f'inversion {forcing.upper()} forcing df', df=True,
                           rewrite=rewrite_files_ok)

        if array_format_ok :
            current_inversion_filename = f"inversion-{forcing.lower()}_{inversion_suffix}.p"

            inversion_array = xxx_df.loc[current_inversion_index].drop('model', axis=1).values
            save_data_file(inversion_array, current_inversion_filename, data_dir=data_dir,
                           label=f'inversion {forcing.upper()} forcing array', rewrite=rewrite_files_ok)

            print(f" inversion {forcing.upper()} array -> dim: {inversion_array.shape}")

elif save_data_ok and save_anc_data_only_ok:
    print(f" ** saving ancillary data only, data arrays not authorized to be saved: set False to `save_anc_data_only_ok` Boolean variables **\n")

else:
    print(f" ** data saving not authorized: set True to `save_data_ok` and False to `save_anc_data_only_ok` Boolean variables **\n")



Ecriture des donnees des forcages:


GHG:
 Saving current inversion GHG forcing df to filename 'inversion-ghg_INVDGM1-S1-NMx100_z48v0_df.p'

AER:
 Saving current inversion AER forcing df to filename 'inversion-aer_INVDGM1-S1-NMx100_z48v0_df.p'

NAT:
 Saving current inversion NAT forcing df to filename 'inversion-nat_INVDGM1-S1-NMx100_z48v0_df.p'


In [25]:
data_dir

'/Users/dahmani/Desktop/stagelong/projetlong/data_source_dr/Region48'

In [30]:
#!(cd /Users/carlos/Clouds/SUnextCloud/Labo/Travaux/Theses-et-stages/These_Constantin/constantin_data/data_source/; ls -lrth {train,test,inversion}-*)
!(cd /Users/dahmani/Desktop/stagelong/projetlong/data_source_dr/Region48; ls -lrth inversion-*)
#!rm /Users/carlos/Clouds/SUnextCloud/Labo/Travaux/Theses-et-stages/These_Constantin/constantin_data/data_source/{train,test}-combi_*
#!rm /Users/carlos/Clouds/SUnextCloud/Labo/Travaux/Theses-et-stages/These_Constantin/constantin_data/data_source/inversion-*_INVDGM1-S1-NMx400_v4*.p

-rw-r--r--@ 1 dahmani  staff    20K Mar  9 15:18 inversion-combi-dic_INVDGM1-S1-NMx100_z48v0.p
-rw-r--r--@ 1 dahmani  staff   1.0M Mar 14 13:59 inversion-ghg_INVDGM1-S1-NMx100_z48v0_df.p
-rw-r--r--@ 1 dahmani  staff   1.0M Mar 14 13:59 inversion-aer_INVDGM1-S1-NMx100_z48v0_df.p
-rw-r--r--@ 1 dahmani  staff   1.0M Mar 14 13:59 inversion-nat_INVDGM1-S1-NMx100_z48v0_df.p


In [31]:
#!ls -lh /Users/carlos/Clouds/SUnextCloud/Labo/Travaux/Theses-et-stages/These_Constantin/constantin_data/data_source

In [32]:
print(f"\nFIN d'Execution:\n  case_out_dir ......... {inversion_suffix}\n  Date/heure d'arret ... {datetime.datetime.now()}\n\n  -> Ready for {n_sim_max_by_mod} n_sim_max_by_mod")


FIN d'Execution:
  case_out_dir ......... INVDGM1-S1-NMx100_z48v0
  Date/heure d'arret ... 2023-03-14 13:59:47.299402

  -> Ready for 100 n_sim_max_by_mod
